In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion

In [2]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y1.csv')
data.describe()

,MemberID,LabCount_total,LabCount_months,DrugCount_total,DrugCount_months,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,...,age_55,age_65,age_75,age_85,age_MISS,SexMALE,sexFEMALE,sexMISS,ClaimsTruncated,TARGET
count,7.603800e+04,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,...,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000
mean,4.994761e+07,6.148886,1.580289,10.118296,3.703661,11.320221,4.404127,3.705805,1.022344,2.145335,...,0.112365,0.116152,0.161130,0.085484,0.059733,0.367803,0.449025,0.183172,0.052224,0.467095
std,2.890145e+07,7.233436,1.814161,15.353455,4.089925,10.480493,3.341382,2.615952,0.150621,1.021198,...,0.315817,0.320410,0.367653,0.279602,0.236994,0.482211,0.497398,0.386810,0.222480,1.612194
min,2.100000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.483719e+07,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,2.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.990721e+07,4.000000,1.000000,3.000000,2.000000,8.000000,3.000000,3.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.510112e+07,10.000000,2.000000,14.000000,7.000000,16.000000,6.000000,5.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,9.999863e+07,89.000000,12.000000,84.000000,12.000000,43.000000,27.000000,19.000000,3.000000,8.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000


In [3]:
data.drop("MemberID", axis = 1, inplace = True)
data.drop("Year", axis = 1, inplace = True)

X = data.loc[:, data.columns != 'TARGET']
y = data['TARGET']
y.describe()

count    76038.000000
mean         0.467095
std          1.612194
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         15.000000
Name: TARGET, dtype: float64

In [4]:
def eval(model, X_test, y_test):
    pred_y = model.predict(X_test) # your predicted y values
    pred_y = pred_y.astype(int)
    # The root mean square logarithmic error
    msle = np.mean( (np.log(pred_y + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    
    print ("RMSLE: %.4f" % rmsle)
    from sklearn.metrics import r2_score
    r2 = r2_score(pred_y, y_test)
    print ("R2 value: %.4f" % r2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=99)

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('regr', Lasso())
])

param_grid = [
    {
        'regr': [Lasso(), Ridge()],
        'regr__alpha': np.logspace(-4, 1, 6),
    },
    {
        'regr': [SGDRegressor()],
        'regr__alpha': np.logspace(-5, 0, 6),
        'regr__max_iter': [500, 1000],
    }
]

grid = GridSearchCV(pipe, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

model.best_params_, model.best_score_
#eval(grid, X_test, y_test)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


In [6]:
eval(grid, X_test, y_test)

Score:	0.0724911703671487


In [7]:
data_submit = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
data_submit.drop("MemberID", axis = 1, inplace = True)
data_submit.drop("Year", axis = 1, inplace = True)

y_submit = data_submit['TARGET']
X_submit = data_submit.loc[:, data_submit.columns != 'TARGET']

eval(grid, X_submit, y_submit)

RMSLE: 0.4891
R2 value: -14.2906
